# 作业 4：多模态数据提取实践

本作业旨在练习不同格式文档（PDF 和 EPUB）的特定内容提取技术。

**任务列表**：
1. **PDF 表格提取**：使用 `pdfplumber` 从 PDF 中精准提取表格数据。
2. **EPUB 图片提取**：从 EPUB 电子书中提取内嵌的图片资源。

In [ ]:
import os
import pdfplumber
import pandas as pd
import zipfile
from IPython.display import display, Image
import io

# 定义文件路径
current_dir = os.getcwd()
# 向上两级找到 asset 目录 (如果当前在 notebooks 目录下)
project_root = os.path.abspath(os.path.join(current_dir, '..'))
asset_dir = os.path.join(project_root, 'asset')

pdf_filename = "billionaires_page-1-5.pdf"
epub_filename = "投资第一课 (孟岩) (Z-Library).epub"

pdf_path = os.path.join(asset_dir, pdf_filename)
epub_path = os.path.join(asset_dir, epub_filename)

print(f"PDF Path: {pdf_path}")
print(f"EPUB Path: {epub_path}")

# 检查文件是否存在
if not os.path.exists(pdf_path):
    print("Warning: PDF file not found!")
if not os.path.exists(epub_path):
    print("Warning: EPUB file not found!")

## 1. PDF 表格提取 (使用 pdfplumber)

我们将读取 PDF 文件，识别其中的表格，并将其转换为 Pandas DataFrame 进行展示。

In [ ]:
if os.path.exists(pdf_path):
    print(f"正在解析: {pdf_filename}...")
    
    with pdfplumber.open(pdf_path) as pdf:
        total_tables = 0
        
        # 遍历每一页
        for i, page in enumerate(pdf.pages):
            # 提取当前页的所有表格
            tables = page.extract_tables()
            
            if tables:
                print(f"\n--- 第 {i+1} 页发现 {len(tables)} 个表格 ---")
                
                for j, table in enumerate(tables):
                    total_tables += 1
                    # 将表格转换为 DataFrame
                    # 假设第一行是表头
                    if len(table) > 1:
                        df = pd.DataFrame(table[1:], columns=table[0])
                    else:
                        df = pd.DataFrame(table)
                    
                    print(f"表格 {j+1} (前5行):")
                    display(df.head())
    
    if total_tables == 0:
        print("在该 PDF 中未检测到表格。")
else:
    print("跳过 PDF 处理：文件不存在")

## 2. EPUB 图片提取

EPUB 文件本质上是一个 ZIP 压缩包。我们可以直接解压它，找到其中的图片文件（通常在 `Images` 或 `OEBPS/images` 目录下）。
这里我们不依赖额外的 `ebooklib` 库，而是使用 Python 内置的 `zipfile` 来实现通用的提取。

In [ ]:
if os.path.exists(epub_path):
    print(f"正在解析: {epub_filename}...")
    
    image_count = 0
    
    try:
        with zipfile.ZipFile(epub_path, 'r') as z:
            # 列出所有文件
            all_files = z.namelist()
            
            # 过滤出图片文件 (jpg, png, jpeg, webp)
            image_files = [f for f in all_files if f.lower().endswith(('.png', '.jpg', '.jpeg', '.webp'))]
            
            print(f"发现 {len(image_files)} 张图片资源。")
            
            # 展示前 5 张图片
            for img_path in image_files[:5]:
                print(f"\n读取图片: {img_path}")
                
                # 读取图片二进制数据
                img_data = z.read(img_path)
                
                # 在 Notebook 中展示
                display(Image(data=img_data, width=300)) # 限制宽度以便展示
                image_count += 1
                
    except zipfile.BadZipFile:
        print("错误：该文件不是有效的 EPUB/ZIP 文件。")
    except Exception as e:
        print(f"处理 EPUB 时出错: {e}")
else:
    print("跳过 EPUB 处理：文件不存在")